## 06-01-00. FAISS와 cuVS

- FAISS : 고차원 벡터들 사이에서 "가장 비슷한 벡터"를 빠르게 찾기 위한 라이브러리
- cuVS : NVIDIA가 만든 GPU 벡터 검색 라이브러리

### Colab Setting

In [15]:
!pip install -q cuvs-cu13 --extra-index-url=https://pypi.nvidia.com

In [16]:
!pip install -q faiss-gpu-cu12

In [17]:
import cuvs
import faiss

cuvs_version = cuvs.__version__
faiss_cpu_version = faiss.__version__

print(f"cuVS version: {cuvs_version}")
print(f"FAISS CPU version: {faiss_cpu_version}")

cuVS version: 26.02.000
FAISS CPU version: 1.13.2


### Data Setting

In [18]:
import numpy as np
import cupy as cp

# 1. NumPy 시드 고정
np.random.seed(42)

# 2. 데이터 생성 : 분포가 다른 2개의 2차원 데이터를 합쳐서 벡터 크기 차이가 나게 만듦
dataset_np = np.hstack([
    np.random.random((50000, 25)),   # [0, 1) 구간의 균일 분포
    np.random.randn(50000, 25) * 5   # 평균 0, 표준편차 5인 정규 분포
]).astype(np.float32)

# 3. Numpy 배열을 CuPy 배열로 변환
dataset_cp = cp.asarray(dataset_np)

# 4. 쿼리 개수 지정 (첫 번째 샘플을 쿼리로 사용)
n_queries = 1
query_np = dataset_np[:n_queries]
query_cp = dataset_cp[:n_queries]

# 5. 데이터셋과 쿼리 벡터의 일부 출력
print("데이터셋 일부:\\n", dataset_np[:1], end='\\n\\n')  # 첫 2개 샘플만 출력
print("쿼리 벡터:\\n", query_np)

데이터셋 일부:\n [[ 0.37454012  0.9507143   0.7319939   0.5986585   0.15601864  0.15599452
   0.05808361  0.8661761   0.601115    0.7080726   0.02058449  0.96990985
   0.83244264  0.21233912  0.18182497  0.1834045   0.30424225  0.52475643
   0.43194503  0.29122913  0.6118529   0.13949387  0.29214466  0.36636186
   0.45606998 -6.9410195  -0.7064649  -2.3158252   8.927271   -0.20885432
  -5.3248634  -2.2789273  -1.2751076   2.4537957   4.761805    5.567778
  -4.7681065   0.42192096  9.123224   -2.6262207   0.5368253  -8.330255
   1.1769367  -2.1636157   0.6136664   1.3033437  -1.3473666   1.8423494
   4.220154   11.622142  ]]\n\n쿼리 벡터:\n [[ 0.37454012  0.9507143   0.7319939   0.5986585   0.15601864  0.15599452
   0.05808361  0.8661761   0.601115    0.7080726   0.02058449  0.96990985
   0.83244264  0.21233912  0.18182497  0.1834045   0.30424225  0.52475643
   0.43194503  0.29122913  0.6118529   0.13949387  0.29214466  0.36636186
   0.45606998 -6.9410195  -0.7064649  -2.3158252   8.927271   -0.2

## 06-01-01. Brute Force

- 모든 벡터를 저장해두고, 검색할 때 전부 다 비교하는 방식 (무차별 대입)

- 거리 계산 종류
  - L2 Distance (Euclidean Distance) : 두 벡터 사이 직선거리 (좌표 차이 계산)
  - Inner Product (내적) : 두 벡터가 얼마나 같은 방향을 보는지 (벡터 크기에 영향을 받음)
  - Cosine Similarity : 두 벡터가 얼마나 같은 방향을 보는지 (벡터 크기 제거)

- 라이브러리 별 인덱스 방식 (거리 계산)
  - FAISS : IndexFlatL2(L2 Distance), IndexFlatIP(Inner Product) -> 코사인 유사도 계산 함수가 없음
  - cuVS : brute force backend (L2 / Inner Product / Cosine 선택 가능)



### 1. FAISS Brute-force

#### Cosine Similarity와 Inner Product의 관계

코사인 유사도 정의:

$$
\cos(\theta) = \frac{q \cdot x}{\|q\| \|x\|}
$$

내적(Inner Product) 정의:

$$
q \cdot x = \sum_{i=1}^{d} q_i x_i
$$

L2 정규화를 적용하면(벡터 길이를 1로 만듦):

$$
\|q\| = 1, \quad \|x\| = 1
$$

따라서,

$$
\cos(\theta) = q \cdot x
$$


즉, 벡터를 L2 정규화한 뒤 Inner Product를 계산하면  
그 값은 Cosine Similarity와 동일하다.

따라서 벡터를 `faiss.normalize_L2()`로 정규화하고  
FAISS에서 `IndexFlatIP`를 사용하면  
Cosine Similarity 기반 검색이 가능하다.

In [19]:
%%time
import numpy as np
import faiss

# 1. 파라미터 설정 : Top-k (가장 유사한 벡터 k개)
k = 10

# 2. 코사인 유사도 계산에 필요한 L2 정규화 적용
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 3. L2 정규화된 데이터를 바탕으로 Brute-force 인덱스 생성
index = faiss.IndexFlatIP(len(dataset_np[0]))

# 4. 인덱스에 벡터 추가
index.add(dataset_np)

# 5. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 6. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])

Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [1.         0.7389719  0.6744807  0.6726349  0.67114633 0.6699053
 0.6660687  0.65731394 0.6533355  0.65039325]
CPU times: user 26.7 ms, sys: 0 ns, total: 26.7 ms
Wall time: 14.7 ms


### 2. cuVS Brute-force

In [20]:
%%time
import cupy as cp
from cuvs.neighbors import brute_force

# 1. 파라미터 설정 : Top-k
k = 10

# 2. 코사인 유사도를 사용한 Brute-force 인덱스 생성
index = brute_force.build(dataset_cp, metric="cosine")

# 3. 벡터 검색
distances, neighbors = brute_force.search(index, query_cp, k)

# 4. CuPy 배열을 Numpy 배열로 변경
distances = cp.asarray(distances)
neighbors = cp.asarray(neighbors)

# 5. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])

Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [-1.1920929e-07  2.6102811e-01  3.2551938e-01  3.2736516e-01
  3.2885367e-01  3.3009470e-01  3.3393139e-01  3.4268612e-01
  3.4666449e-01  3.4960675e-01]
CPU times: user 2.95 ms, sys: 0 ns, total: 2.95 ms
Wall time: 2.81 ms


## 06-01-02. IVF-Flat

- nverted File Flat (역 파일 인덱스) : 비슷한 군집 몇개만 탐색하는 방식
- 방식 요약
  1. 벡터 공간을 여러개의 군집으로 나눔
  2. 쿼리와 가까운 n개의 군집 내에서 Brute Force (선형 탐색)


### 1. FAISS IVF-Flat

In [21]:
%%time
import numpy as np
import faiss

# 1. 코사인 유사도 계산에 필요한 L2 정규화 적용
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 2. 파라미터 설정 : Top-k, 클러스터 개수, 탐색할 클러스터 개수, 벡터 차원
k = 10
n_list = 1000
n_probes = 500
d = dataset_np.shape[1]

# 3. IVF-Flat 인덱스 생성 (내부는 내적 유사도)
quantizer = faiss.IndexFlatIP(d)   # 군집 중심용 인덱스
index = faiss.IndexIVFFlat(quantizer, d, n_list, faiss.METRIC_INNER_PRODUCT)

# 4. 인덱스 학습
index.train(dataset_np)

# 5. 인덱스에 벡터 추가
index.add(dataset_np)

# 6. 탐색할 군집 수 설정
index.nprobe = n_probes

# 7. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 8. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])

Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [1.         0.73897195 0.67448074 0.6726349  0.6711464  0.6699053
 0.66606873 0.65731394 0.6533355  0.65039325]
CPU times: user 2.24 s, sys: 2.41 ms, total: 2.24 s
Wall time: 2.3 s


### 2. cuVS IVF-Flat

In [22]:
%%time
import cupy as cp
from cuvs.neighbors import ivf_flat
from cuvs.neighbors.ivf_flat import IndexParams, SearchParams

# 1. 파라미터 설정 : Top-k, 클러스터 개수, 탐색할 클러스터 개수
k = 10
n_list = 1000
n_probes = 500

# 2. 인덱스 생성 파라미터 정의 (코사인 유사도 + 클러스터 개수)
params = IndexParams(
    n_lists=n_list,
    metric="cosine"
)

# 3. IVF-Flat 인덱스 생성
index = ivf_flat.build(params, dataset_cp)

# 4. 검색 파라미터 정의
search_params = SearchParams(n_probes=n_probes)

# 5. 벡터 검색
distances, neighbors = ivf_flat.search(search_params, index, query_cp, k)

# 6. CuPy 배열을 NumPy 배열로 변환
distances = cp.asnumpy(distances)
neighbors = cp.asnumpy(neighbors)

# 7. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])


Top-k 인덱스: [    0 48906 22098 20005 43820 21295 17192  6777 14367  7147]
유사도 점수 (cosine): [0.         0.2610281  0.32551938 0.32736522 0.32885373 0.3300947
 0.33393133 0.34268612 0.3466646  0.34960675]
CPU times: user 212 ms, sys: 10 ms, total: 222 ms
Wall time: 406 ms


## 06-01-03. IVF-PQ

- IVF-Flat은 군집 선택은 빠르지만 여전히 원본 벡터를 그대로 저장해 메모리를 많이 사용
- 벡터를 압축해서 저장하자는 방식 -> Product Quantization

#### IVF-PQ 방식 요약

1. 벡터 공간을 여러 개의 군집으로 나눔 (IVF 단계)
    1. 전체 벡터에 대해 K-means 수행
    2. K개의 coarse cluster(상위 군집) 생성
    3. 각 벡터를 가장 가까운 군집에 할당
    4. 군집 중심(centroid)을 저장
2. 벡터를 압축하여 저장 (PQ 단계)
    1. 전체 벡터를 m개의 조각(Subspace)으로 분할
        - 각 조각의 차원은 d / m
    2. 각 조각에 해당하는 차원만 모아 데이터 구성
    3. 각 조각에 대해 K-means 수행
        - 클러스터 중심(centroid) 생성
    4. 각 조각마다 코드북(Codebook) 생성
        - 총 m개의 코드북 생성
    5. 각 벡터의 조각을 가장 가까운 centroid의 인덱스로 치환
        - 결과적으로 벡터는 길이 m의 정수 인덱스 벡터(PQ 코드)로 저장됨
3. 탐색 수행
    1. 검색하고자 하는 벡터와 K개의 군집 중심 간 거리 계산
    2. 가장 가까운 n_probes개의 군집 선택
    3. 선택된 군집 내부 벡터들에 대해:
        1. 검색하고자 하는 벡터를 m개의 조각으로 분할
        2. 각 조각과 코드북 centroid 간 거리 계산
        3. Distance Lookup Table 생성
        4. 데이터 벡터의 PQ 인덱스를 사용해 테이블 값을 합산
        5. 전체 거리의 근사값 계산
    4. Top-k 결과 반환

- 기존 데이터는 압축 / 쿼리(검색 벡터)는 비압축
  - 비대칭 거리 계산 ADC 방식을 사용 : 쿼리는 원본, 데이터는 압축 → 정확도 비교적 높음
  - SDC를 사용하면 쿼리도 압축, 데이터도 압축 → 더 빠르지만 정확도 낮음

#### 핵심 요약

- IVF: 탐색 범위를 줄이는 단계
- PQ: 벡터를 압축하여 저장하는 단계
- IVF-PQ: 탐색 범위 축소 + 압축 기반 근사 거리 계산을 결합한 알고리즘

### 4. FAISS IVF-PQ

In [23]:
%%time
import numpy as np
import faiss

# 1. 코사인 유사도 계산을 위한 L2 정규화
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 2. 파라미터 설정
k = 10
n_list = 1000
n_probes = 500
d = dataset_np.shape[1]

# 3. IVF-PQ 인덱스 생성
quantizer = faiss.IndexFlatIP(d)  # 군집 중심용 인덱스
m = 10                            # PQ의 서브벡터 개수 (열 50차원의 약수)
nbits = 8                         # 각 서브벡터를 2^nbits개의 centroid로 압축
index = faiss.IndexIVFPQ(
    quantizer, d, n_list, m, nbits, faiss.METRIC_INNER_PRODUCT
)

# 4. 인덱스 학습
index.train(dataset_np)

# 5. 인덱스에 벡터 추가
index.add(dataset_np)

# 6. 탐색할 군집 수 설정
index.nprobe = n_probes

# 7. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 8. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])


Top-k 인덱스: [    0 48906 27836 43820 32021 22098 31260 10781 20526 34583]
유사도 점수 (cosine): [0.81631005 0.73726285 0.7087823  0.6927892  0.6907227  0.6865624
 0.6560866  0.65572387 0.6474889  0.64162534]
CPU times: user 8.72 s, sys: 16.4 ms, total: 8.74 s
Wall time: 12.5 s


### 5. cuVS IVF-PQ


In [24]:
%%time

import cupy as cp
from cuvs.neighbors import ivf_pq
from cuvs.neighbors.ivf_pq import IndexParams, SearchParams

# 1. 파라미터 설정 : Top-k, 클러스터 개수, 탐색할 클러스터 개수
k = 10
n_list = 1000
n_probes = 500

# 2. 인덱스 생성 파라미터 정의 (코사인 유사도, 클러스터 개수 PQ 파라미터)
params = IndexParams(
    n_lists=n_list,    # 클러스터 개수 지정
    metric="cosine",   # 인덱스 생성에 사용하는 거리는 코사인 유사도
    pq_bits=8,         # 각 코드북 인덱스 비트수 (8비트)
    pq_dim=10          # PQ 조각 수 (열 50차원의 약수)
)

# 3. IVF-PQ 인덱스 생성
index = ivf_pq.build(params, dataset_cp)

# 4. 검색 파라미터 정의
search_params = SearchParams(n_probes=n_probes)

# 5. 벡터 검색
distances, neighbors = ivf_pq.search(search_params, index, query_cp, k)

# 6. CuPy 배열을 NumPy 배열로 변환
distances = cp.asnumpy(distances)
neighbors = cp.asnumpy(neighbors)

# 7. 검색 결과 출력
print("Top-k 인덱스:", neighbors[0])
print("유사도 점수 (cosine):", distances[0])



Top-k 인덱스: [    0 48906 27836 26347 22098 14367 13753 21295 36950 28453]
유사도 점수 (cosine): [0.02722573 0.22007954 0.22070134 0.26345307 0.2948786  0.31555426
 0.31680876 0.31853807 0.3292637  0.3414443 ]
CPU times: user 300 ms, sys: 42.2 ms, total: 343 ms
Wall time: 342 ms


## 06-01-04. 그래프 기반 알고리즘 (HNSW, CAGRA)

### 1. HNSW

![](https://static.wikidocs.net/images/page/281671/HNSW.png)

- 가장 위 레이어에서 시작해 대략 가까운 방향으로 이동.
- 아래 레이어로 내려감. 점점 정밀하게 탐색하고 최하위 레이어에서 최종 근접 이웃 결정

### 2. CAGRA

![](https://static.wikidocs.net/images/page/281671/CAGRA.png)

1.	초기 단일 그래프 생성
    -	각 노드에 많은 이웃 연결
2.	간선 중요도 평가
3.	덜 중요한 간선 제거
4.	최종 최적화된 그래프 생성

#### Data Setting

In [25]:
import numpy as np
import cupy as cp

# 1. NumPy 시드 고정
np.random.seed(42)

# 2. 데이터 생성 : 분포가 다른 2개의 2차원 데이터를 합쳐서 벡터 크기 차이가 나게 만듦
dataset_np = np.hstack([
    np.random.random((500000, 128)),   # [0, 1) 구간의 균일 분포
    np.random.randn(500000, 128) * 5   # 평균 0, 표준편차 5인 정규 분포
]).astype(np.float32)

# 3. Numpy 배열을 CuPy 배열로 변환
dataset_cp = cp.asarray(dataset_np)

# 4. 쿼리 개수 지정
n_queries = 1000
query_np = dataset_np[:n_queries]
query_cp = dataset_cp[:n_queries].copy()

# 5. 데이터셋과 쿼리 벡터의 일부 출력
print("데이터셋 일부:\n", dataset_np[:1], end='\n\n')  # 첫 2개 샘플만 출력
print("쿼리 벡터:\n", query_np)

데이터셋 일부:
 [[ 3.74540120e-01  9.50714290e-01  7.31993914e-01  5.98658502e-01
   1.56018645e-01  1.55994520e-01  5.80836125e-02  8.66176128e-01
   6.01114988e-01  7.08072603e-01  2.05844939e-02  9.69909847e-01
   8.32442641e-01  2.12339118e-01  1.81824967e-01  1.83404505e-01
   3.04242253e-01  5.24756432e-01  4.31945026e-01  2.91229129e-01
   6.11852884e-01  1.39493868e-01  2.92144656e-01  3.66361856e-01
   4.56069976e-01  7.85175979e-01  1.99673787e-01  5.14234424e-01
   5.92414558e-01  4.64504138e-02  6.07544839e-01  1.70524120e-01
   6.50515929e-02  9.48885560e-01  9.65632021e-01  8.08397353e-01
   3.04613769e-01  9.76721123e-02  6.84233010e-01  4.40152496e-01
   1.22038238e-01  4.95176911e-01  3.43885198e-02  9.09320414e-01
   2.58779973e-01  6.62522256e-01  3.11711073e-01  5.20068049e-01
   5.46710253e-01  1.84854463e-01  9.69584644e-01  7.75132835e-01
   9.39498961e-01  8.94827366e-01  5.97899973e-01  9.21874225e-01
   8.84925053e-02  1.95982859e-01  4.52272892e-02  3.25330317e-01


### 3. FAISS HNSW

In [26]:
%%time
import numpy as np
import faiss

# 1. 코사인 유사도 계산을 위한 L2 정규화
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 2. 파라미터 설정
k = 10                               # Top-k
d = dataset_np.shape[1]              # 특성 차원(열 개수)
M = 32                               # 그래프에서 각 노드가 연결할 최대 이웃 수
ef_search = 128                      # 탐색 시 고려할 후보 개수
metric = faiss.METRIC_INNER_PRODUCT  # L2 정규화된 벡터 간 내적으로 코사인 유사도 계산

# 3. HNSW 인덱스 생성
index = faiss.IndexHNSWFlat(d, M, metric)

# 4. 인덱스에 벡터 추가
index.add(dataset_np)

# 5. 탐색 성능 파라미터 설정
index.hnsw.efSearch = ef_search

# 6. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 7. 10개의 쿼리의 검색 결과 출력
for i in range(len(query_np)):
    print(f"\n쿼리 {i+1}:")
    print("Top-k 인덱스:", neighbors[i])
    print("유사도 점수 (cosine):", distances[i])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Top-k 인덱스: [     0 499700 129395  76297 101190 337420 469980 388127 300877  52451]
유사도 점수 (cosine): [1.0000001  0.4143618  0.37672573 0.3729911  0.35840312 0.35734993
 0.35666043 0.34697157 0.34501845 0.34199697]

쿼리 2:
Top-k 인덱스: [     1 460724  24702  11058 327550 178455 302349 230268 102721 350943]
유사도 점수 (cosine): [1.0000001  0.44864494 0.40347752 0.3744548  0.36929336 0.33978468
 0.3390727  0.33360958 0.33265406 0.3322565 ]

쿼리 3:
Top-k 인덱스: [     2 494638 323213 283080 235398 451665 347398  54973  20733 489187]
유사도 점수 (cosine): [1.         0.39540836 0.38864923 0.36930817 0.36460614 0.36345527
 0.36291653 0.36190265 0.36019927 0.3593589 ]

쿼리 4:
Top-k 인덱스: [     3 178740 251784  85997 367027 430017 295468  91930 493251 430371]
유사도 점수 (cosine): [1.0000001  0.37751222 0.37387013 0.36699665 0.34715977 0.34518477
 0.3363788  0.32969585 0.32668194 0.32368952]

쿼리 5:
Top-k 인덱스: [     4 288098 248351  35574 343524 234845 329277  56398 209884 278808]
유

### 4. cuVS CAGRA

In [27]:
def l2_normalize_cp(x, axis=1, eps=1e-12):
    norm = cp.linalg.norm(x, axis=axis, keepdims=True)
    return x / (norm + eps)

In [30]:
%%time
import cupy as cp
from cuvs.neighbors import cagra

# 1. 파라미터 설정
k = 10                               # Top-k
M = 64                               # 그래프에서 각 노드가 연결할 최대 이웃 수
ef_search = 256                      # 탐색 시 고려할 후보 개수

# 2. 코사인 유사도 계산을 위한 L2 정규화
normalized_dataset_cp = l2_normalize_cp(dataset_cp)
normalized_query_cp = l2_normalize_cp(query_cp)

# 3. CAGRA 인덱스 생성 파라미터 설정 (L2 거리 기반)
build_params = cagra.IndexParams(
    metric="inner_product",
    graph_degree=M
    )

# 4. 인덱스 빌드
index = cagra.build(build_params, normalized_dataset_cp)

# 5. 벡터 검색
search_params = cagra.SearchParams(itopk_size=ef_search)
distances, neighbors = cagra.search(search_params, index, normalized_query_cp, k)

# 6. 1000개의 쿼리의 검색 결과 출력
neighbors_np = cp.asarray(neighbors).get()
distances_np = cp.asarray(distances).get()

for i in range(neighbors_np.shape[0]):
    print(f"\n쿼리 {i}:")
    print("Top-k 인덱스:", neighbors_np[i])
    print("유사도 점수 (cosine):", distances_np[i])


쿼리 0:
Top-k 인덱스: [     0 499700 265723  76297 163835 434104  63911  97660 106117 300877]
유사도 점수 (cosine): [0.9999999  0.4143617  0.38176888 0.37299109 0.35987374 0.3521974
 0.34877372 0.34679732 0.34559825 0.34501842]

쿼리 1:
Top-k 인덱스: [     1 460724  24702  11058 119095 327550 400222 423705 391519 131027]
유사도 점수 (cosine): [1.         0.44864494 0.40347752 0.3744547  0.3698097  0.3692933
 0.35941368 0.358553   0.3545121  0.35239   ]

쿼리 2:
Top-k 인덱스: [     2 494638 323213 105816 223728 283080 451665 347398 368153 369591]
유사도 점수 (cosine): [1.         0.3954084  0.38864928 0.38268462 0.37833053 0.3693082
 0.36345527 0.3629166  0.3613519  0.35301393]

쿼리 3:
Top-k 인덱스: [     3 178740  26058 251784 213422 431259 230842  33377 367027 472486]
유사도 점수 (cosine): [1.         0.37751222 0.3744778  0.37387013 0.37174788 0.36836845
 0.35584098 0.35115236 0.34715974 0.34698558]

쿼리 4:
Top-k 인덱스: [     4 288098 122011 138005 242418 398628 381460 428799 248351  23142]
유사도 점수 (cosine): [0.99999994 0.39